In [1]:
import sleqp

In [2]:
import numpy as np

# Solving an unconstrained problem

In the following, we solve the Rosenbrock problem using the standard API rather than the `scipy`-based interface. This offers more fine-grained control over the solution process.

Since SLEQP uses the `logging` framework, we begin by setting up basic logging.

In [3]:
import logging

logging.basicConfig(level=logging.INFO)

Problems are given in terms of a function object, which must provide a set of callbacks to evaluate

- The optimization objective $f(x)$
- The optimization gradient $\nabla f(x)$
- The constraint values $c(x)$
- The constraint Jacobian $J_c(x)$
- The Hessian of the Lagrangian of the problem, given by $L(x, \lambda) := f(x) + \langle c(x), \lambda \rangle$, with respect to the primal variables $x$

Since we are solving an unconstrained problem, there is no need to provide the constraint-related functions in this case.

In [4]:
class RosenbrockFunc:

  def __init__(self):
    self.a = 1.
    self.b = 100.

  # Set the (primal) value
  def set_value(self, v, _):
    self.v = v

  # Return objective value at current value
  def obj_val(self):
    [x, y] = self.v
    (a, b) = (self.a, self.b)

    xsq = x**2

    return (a - x)**2 + b*(y - xsq)**2

  # Return objective gradient at current value
  def obj_grad(self):
    [x, y] = self.v
    (a, b) = (self.a, self.b)

    xsq = x**2

    g = np.array([(4*b*x*(xsq - y)) + 2*x - 2*a,
                  -2*b*(xsq - y)])

    return g

  # Unconstrained problem, cons_val / cons_jac not needed

  # Return constraint values at current value
  # def cons_vals(self):
  #   return np.zeros((num_constraints,))

  # Return constraint Jacobian at current value
  # def cons_jac(self):
  #   return np.zeros((num_constraints, num_variables))

  # Return product of the Hessian of the Lagrangian
  # with the provided multipliers with the given direction
  def hess_prod(self, direction, duals):
    [x, y] = self.v
    (a, b) = (self.a, self.b)
    [dx, dy] = direction

    xsq = x**2

    product = np.array([((8.*b*xsq + 4.*b*(xsq - y) + 2.)*dx - (4.*b*x)*dy),
                        ((-4.*b*x)*dx + (2.*b)*dy)])

    return product


In [5]:
var_lb = np.array([-np.inf, -np.inf])
var_ub = np.array([np.inf, np.inf])

In [6]:
func = RosenbrockFunc()

In [7]:
problem = sleqp.Problem(func,
                        var_lb,
                        var_ub)

In [8]:
x0 = np.array([0., 0.])

In [9]:
solver = sleqp.Solver(problem, x0)

In [10]:
solver.solve()

INFO:sleqp:Solving a problem with 2 variables, 0 constraints, 0 Jacobian nonzeros
INFO:sleqp: Iteration |          Merit  val |       Obj val |      Feas res |      Comp res |      Stat res |       Penalty |   Working set |         LP tr |        EQP tr |   Primal step |     Dual step |          Step type
INFO:sleqp:         0 |    1.0000000000e+00 |  1.000000e+00 |  0.000000e+00 |               |               |  1.000000e+01 |               |               |               |               |               |                   
INFO:sleqp:         1 |    1.0000000000e+00 |  1.000000e+00 |  0.000000e+00 |  0.000000e+00 |  2.000000e+00 |  1.000000e+01 |            -- |  5.656854e-01 |  1.000000e+00 |  1.000000e+00 |  0.000000e+00 |           Rejected
INFO:sleqp:         2 |    1.0000000000e+00 |  1.000000e+00 |  0.000000e+00 |  0.000000e+00 |  2.000000e+00 |  1.000000e+01 |            -- |  5.000000e-01 |  5.000000e-01 |  5.000000e-01 |  0.000000e+00 |           Rejected
INFO:sleqp:       

We can retrieve the solution from the solver:

In [11]:
solver.status

<Status.Optimal: 2>

In [12]:
solution = solver.solution

In [13]:
solution.primal

array([0.99999998, 0.99999995])

In [14]:
solution.obj_val

8.04584380003903e-16

## Settings

The behavior of `SLEQP` can be affected by passing settings to the problem to be used by the solver. As an example, we can solve the problem with a tighter stationarity requirement:

In [15]:
settings = sleqp.Settings(stat_tol=1e-10)

problem = sleqp.Problem(func,
                        var_lb,
                        var_ub,
                        settings=settings)

In [16]:
solver = sleqp.Solver(problem, x0)

In [17]:
solver.solve()

INFO:sleqp:Solving a problem with 2 variables, 0 constraints, 0 Jacobian nonzeros
INFO:sleqp: Iteration |          Merit  val |       Obj val |      Feas res |      Comp res |      Stat res |       Penalty |   Working set |         LP tr |        EQP tr |   Primal step |     Dual step |          Step type
INFO:sleqp:         0 |    1.0000000000e+00 |  1.000000e+00 |  0.000000e+00 |               |               |  1.000000e+01 |               |               |               |               |               |                   
INFO:sleqp:         1 |    1.0000000000e+00 |  1.000000e+00 |  0.000000e+00 |  0.000000e+00 |  2.000000e+00 |  1.000000e+01 |            -- |  5.656854e-01 |  1.000000e+00 |  1.000000e+00 |  0.000000e+00 |           Rejected
INFO:sleqp:         2 |    1.0000000000e+00 |  1.000000e+00 |  0.000000e+00 |  0.000000e+00 |  2.000000e+00 |  1.000000e+01 |            -- |  5.000000e-01 |  5.000000e-01 |  5.000000e-01 |  0.000000e+00 |           Rejected
INFO:sleqp:       